Load scores

In [1]:
import os
import numpy as np
from music21 import converter, pitch, interval, instrument, note, stream, meter, percussion
from matplotlib import pyplot as plt

from xen.visualise import plotPart
from xen.data.SongData import SongDataSet
from xen.data.Filter import NameFilter, SongDataFilter

# paths = ["../../../ai/trainingdata/music/800000_Drum_Percussion_MIDI_Archive/Rock-Indie"]
paths = [
    "../../../ai/trainingdata/music/800000_Drum_Percussion_MIDI_Archive/Superior Drummer 2 Drum Midi/00201@BREAKBEAT HOUSE", 
    "../../../ai/trainingdata/music/800000_Drum_Percussion_MIDI_Archive/Superior Drummer 2 Drum Midi/00201@BREAKS & SWINGBEATS VOL1"
]
# paths = ["../../../ai/trainingdata/music/songsterr/lorna-shore/Lorna Shore - King Ov Deception (552149).mid"]
# paths = ["../../../ai/trainingdata/music/800000_Drum_Percussion_MIDI_Archive/50's Drummer MIDI Files/01 Rock'n'Roll/01 Dancin Rick 166BPM/01 8th Hat.mid"]

dataset = SongDataSet.fromMidiPaths(paths, recursive=False)


Loading 52 files


IntProgress(value=0, max=52)

Label(value='')

Loaded 52 songs
Loading 69 files


IntProgress(value=0, max=69)

Label(value='')

Loaded 121 songs


## Filter

In [3]:
dataset = dataset.filterTimeSig('4/4')

print(f'{len(dataset.songs)} songs after filtering')

Filtered to 121 songs
121 songs after filtering


## Parts analysis

In [ ]:
partscount = {}
partnames = {}
instruments = {}
scorepartcounts = {}

for i, song in enumerate(dataset.songs):
    # parts = song.getParts()
    parts = song.getPartsByInstruments(instrumentFilter = NameFilter(include = "Percussion"))

    partscount[len(parts)] = partscount.get(len(parts), 0) + 1
    scorepartcounts[song.filePath] = len(parts)

    # instrument.partitionByInstrument(song.score)
    # print(song.score.getInstruments())

    for part in parts:
        partnames[part.partName] = partnames.get(part.partName, 0) + 1
        partinstrument = part.getInstruments()[-1]
        # print(part.getInstruments()[-1].instrumentName)
        # print(part.getInstruments()[-1].midiProgram)
        # print(part.getInstruments()[-1].midiChannel)
        instruments[partinstrument.instrumentName] = instruments.get(partinstrument.instrumentName, 0) + 1

# Analysis of number of scores with different number of parts
print("Part counts")
print(partscount)
# List unique part names
print("Part names")
print(partnames)
# List unique instrument names
print("Instrument names")
print(instruments)
# List scores with number of parts
print("Score part counts")
print(sorted(scorepartcounts.items(), key=lambda x: x[1], reverse=True))

## Time Signatures

Count number of scores which use each time signature

In [ ]:
timesigs = dataset.splitByTimeSignature()

for timesig in timesigs:
    print(f'{len(timesigs[timesig].songs)} songs in {timesig} time')

# print(timesigs['3/4'].songs[0].filePath)

In [ ]:
# time signatures are usually stored within the measures object and can change mid score
timesigscount = dict()
scoretimesigscount = dict()

for i, song in enumerate(dataset.songs):
    timesigs = song.score.recurse().getElementsByClass(meter.TimeSignature)
    scoretimesigscount[song.filePath] = len(timesigs)

    for timesig in timesigs:
        timsiglabel = f'{timesig.numerator}/{timesig.denominator}'
        timesigscount[timsiglabel] = timesigscount.get(timsiglabel, 0) + 1

print(timesigscount)
print(sorted(scoretimesigscount.items(), key=lambda x: x[1], reverse=True))

## Notes Used

In [5]:
from xen.codecs.NoteSequenceSparseCodec import NoteSequenceSparseCodec
from xen.data.SongData import elementToMidiPitches
from xen.data.PercussionMap import getPercussionName
from music21.percussion import PercussionChord
from music21.note import Note, Unpitched, NotRest
from music21.chord import Chord
from music21.stream.base import Part
import json

from typing import List

notecounts = {}

for song in dataset.songs:
    songnotecounts = {}
    # for part in song.getParts():
    for part in song.getPartsByInstruments(NameFilter(include=["Percussion"])):
        for element in part.recurse().notes:
            for midipitch in elementToMidiPitches(element):
                notecounts[midipitch] = notecounts.get(midipitch, 0) + 1
                songnotecounts[midipitch] = songnotecounts.get(midipitch, 0) + 1
    songnotecounts = dict(sorted(songnotecounts.items()))
    songnotecounts = {getPercussionName(k): v for k, v in songnotecounts.items()}
    # print(song.filePath)
    # print(json.dumps(songnotecounts, indent=4))

notecounts = dict(sorted(notecounts.items()))

# convert midi pitches to drum names
notecounts = {getPercussionName(k): v for k, v in notecounts.items()}
# print(json.dumps(notecounts, indent=4))

# order dict by values
notecounts = dict(sorted(notecounts.items(), key=lambda x: x[1], reverse=True))

print(json.dumps(notecounts, indent=4))


{
    "CLOSED_HI_HAT": 3967,
    "BASS_DRUM_1": 2381,
    "60": 2111,
    "ACCOUSTIC_SNARE": 1839,
    "TAMBOURINE": 744,
    "80": 644,
    "81": 539,
    "ELECTRIC_SNARE": 494,
    "62": 445,
    "64": 397,
    "OPEN_HI_HAT": 343,
    "PEDAL_HI_HAT": 310,
    "COWBELL": 306,
    "61": 186,
    "63": 185,
    "70": 136,
    "SIDE_STICK": 88,
    "ACCOUSTIC_BASS_DRUM": 11,
    "CRASH_CYMBAL_1": 10,
    "CRASH_CYMBAL_2": 8,
    "HIGH_FLOOR_TOM": 6,
    "LOW_FLOOR_TOM": 5,
    "LOW_TOM": 5,
    "LOW_MID_TOM": 5,
    "HI_MID_TOM": 5
}
